# 计算机视觉基础之opencv框架和插值法

## 1.图像处理常用操作
- 平移变换
- 旋转变换
- 放缩变换
这些几何变换并不改变图象的象素值，只是在图象平面上进行象素的重新排列。

## 2.常见的基础概念
- 灰度值：灰度是表明图像明暗的数值，即黑白图像中点的颜色深度，范围一般从0到255，白色为255 ，黑色为0，故黑白图片也称灰度图像。灰度值指的是单个像素点的亮度。灰度值越大表示越亮。

- 像素点：像素点是最小的图像单元，一张图片由好多的像素点组成

- RGB:因为一个像素点的颜色是由RGB三个值来表现的，所以像素点矩阵对应三个颜色向量矩阵，分别是R矩阵(500 *338大小)，G矩阵(500 *338大小)，B矩阵(500 *338大小)。如果每个矩阵的第一行第一列的值分别为：R：240，G：223，B：204，所以这个像素点的颜色就是（240,223,204）

## 3.插值法
- 插值法用于解决上述图像处理的常用操作中，因为输入图像的灰度值仅在整数位置有定义，但输出图像的灰度值一般由处在非整数坐标上的(u,v)值决定。

- 常见的插值法：最近邻插值、双线性插值和三次样条插值

### 最近邻插值算法
#### 原理介绍
假设
- 源图像A的尺寸$m\times n$
- 经过最近邻插值处理后图像B的尺寸为$a \times b$

缩放倍数
$$K=\frac{a}{m}=\frac{b}{n}$$
取B图像中一点$(x_0,y_0)$，得到对应在A图像中的理论计算位置为
$$(\frac{x_0 \times m}{a},\frac{y_0 \times n}{b})$$
由于图像A中存在的位置理论数值都为整数，但理论计算位置可能为小数，则代表B图像在A图像中无实际对应点，此时对其**四舍五入**，即是把A图像中距离该理论点最近的一个的点当作它，即为**最近邻插值**。该方法对应于OpenCV坐标中则是寻找距离最近的点。
#### 缺点
用该方法作放大处理时，在图象中可能出现明显的块状效应
#### 代码

In [ ]:
import numpy as np
import math 
def nearest_neighbor(input_signal,zoom_multiples):
    '''
    最近邻插值（适用于灰度图）
    :param input_signal: 输入图像
    :param zoom_multiples:  缩放倍数
    :return: 缩放后的图像
    '''
    input_signal_cp=np.copy(input_signal)#输入图像的副本
    
    input_row,input_col=input_signal_cp.shape#获取图像的行、列
    
    #输出图像的尺寸
    output_row=int(input_row*zoom_multiples)
    output_col=int(input_col*zoom_multiples)
    
    #输出缩放后的图片，空值为0
    output_signal=np.zeros((output_row,output_col))
    
    for i in range(output_row):
        for j in range(output_col):
            #输出图片中坐标 （i，j）对应至输入图片中的（m，n）
            m=round(i/output_row*input_row)
            n=round(j/output_col*input_col)
            #防止四舍五入后越界
            if m >= input_row:
                m = input_row - 1
            if n >= input_col:
                n = input_col - 1
            # 插值
            output_signal[i, j] = input_signal_cp[m, n]
    return output_signal

### 双线性插值算法原理介绍
#### 原理介绍
与最近领插值法类似，首先同样的步骤根据放大倍数与缩放前后的图像尺寸找到B图像中对应A图像的坐标，找到最近的4个点，根据双线性插值法的表达式计算该点，即完成双线性插值。

线性插值：
$$ y=y_{0}+\left(x-x_{0}\right) \frac{y_{1}-y_{0}}{x_{1}-x_{0}}=y_{0}+\frac{\left(x-x_{0}\right) y_{1}-\left(x-x_{0}\right) y_{0}}{x_{1}-x_{0}} $$

双线性插值就是线性插值在二维时的推广,在两个方向上做三次线性插值，具体操作如下图所示：
![avatar](双线性插值原理图.png)


令$f(x，y)$为两个变量的函数，其在单位正方形顶点的值已知。假设我们希望通过插值得到正方形内任意点的函数值。则可由双线性方程: $$ f(x, y)=a x+b y+c x y+d $$
来定义的一个双曲抛物面与四个已知点拟合。
首先对上端的两个顶点进行线性插值得：

$$ f(x, 0)=f(0,0)+x[f(1,0)-f(0,0)] $$

类似地，再对底端的两个顶点进行线性插值有： $$ f(x, 1)=f(0,1)+x[f(1,1)-f(0,1)] $$

最后，做垂直方向的线性插值，以确定：

$$ f(x, y)=f(x, 0)+y[f(x, 1)-f(x, 0)] $$

整理得：

$$ \begin{array}{l} f(x, y)=[f(1,0)-f(0,0)] x+[f(0,1)-f(0,0)] y \ +[f(1,1)+f(0,0)-f(0,1)-f(1,0)] x y+f(0,0) \end{array} $$
#### 与最近邻插值法比较
相对于最近邻插值简单的四舍五入，双线性插值的处理更为科学，优化了边缘保护
#### 代码

In [ ]:
import numpy as np
import math

def double_linear(input_signal, zoom_multiples):
    '''
    双线性插值
    :param input_signal: 输入图像
    :param zoom_multiples: 放大倍数
    :return: 双线性插值后的图像
    '''
    input_signal_cp = np.copy(input_signal)   # 输入图像的副本

    input_row, input_col = input_signal_cp.shape # 输入图像的尺寸（行、列）

    # 输出图像的尺寸
    output_row = int(input_row * zoom_multiples)
    output_col = int(input_col * zoom_multiples)

    output_signal = np.zeros((output_row, output_col)) # 输出图片

    for i in range(output_row):
        for j in range(output_col):
            # 输出图片中坐标 （i，j）对应至输入图片中的最近的四个点点（x1，y1）（x2, y2），（x3， y3），(x4，y4)的均值
            temp_x = i / output_row * input_row
            temp_y = j / output_col * input_col

            x1 = int(temp_x)
            y1 = int(temp_y)

            x2 = x1
            y2 = y1 + 1

            x3 = x1 + 1
            y3 = y1

            x4 = x1 + 1
            y4 = y1 + 1

            u = temp_x - x1
            v = temp_y - y1

            # 防止越界
            if x4 >= input_row:
                x4 = input_row - 1
                x2 = x4
                x1 = x4 - 1
                x3 = x4 - 1
            if y4 >= input_col:
                y4 = input_col - 1
                y3 = y4
                y1 = y4 - 1
                y2 = y4 - 1

            # 插值
            output_signal[i, j] = (1-u)*(1-v)*int(input_signal_cp[x1, y1]) + (1-u)*v*int(input_signal_cp[x2, y2]) + u*(1-v)*int(input_signal_cp[x3, y3]) + u*v*int(input_signal_cp[x4, y4])
    return output_signal

## 4.基于OpenCV的实现
#### cv2函数原型
‘cv2.resize(src, dsize[, dst[, fx[, fy[, interpolation]]]])’
#### 参数：
| 参数        | 描述          | 
| ------------- |:-------------:| 
| src     | 【必需】原图像 | 
| dsize      | 【必需】输出图像所需大小|
| fx | 【可选】沿水平轴的比例因子|
|fy  | 【可选】沿垂直轴的比例因子|
|interpolation|【可选】插值方式|

#### 插值方式：
|        |         | 
| ------------- |:-------------:| 
|cv.INTER_NEAREST    | 最近邻插值 | 
|cv.INTER_LINEAR     | 双线性插值|
|cv.INTER_CUBIC| 基于4x4像素邻域的3次插值法|
|cv.INTER_AREA | 基于局部像素的重采样|

注：通常，缩小使用cv.INTER_AREA，放缩使用cv.INTER_CUBIC(较慢)和cv.INTER_LINEAR(较快效果也不错)。默认情况下，所有的放缩都使用cv.INTER_LINEAR。


In [ ]:
import cv2
 
if __name__ == "__main__":
    img = cv2.imread('imag1.jpeg', cv2.IMREAD_UNCHANGED)
    
    print('Original Dimensions : ',img.shape)
    
    scale_percent = 30       # percent of original size
    width = int(img.shape[1] * scale_percent / 100)#读取缩放后的列
    height = int(img.shape[0] * scale_percent / 100)#读取缩放后的行
    dim = (width, height)
    # resize image
    resized = cv2.resize(img, dim, interpolation = cv2.INTER_LINEAR)#使用双线性插值法

    fx = 1.5
    fy = 1.5

    resized1 = cv2.resize(resized, dsize=None, fx=fx, fy=fy, interpolation = cv2.INTER_NEAREST)#最近邻插值法
    
    resized2 = cv2.resize(resized, dsize=None, fx=fx, fy=fy, interpolation = cv2.INTER_LINEAR)#双线性插值法
    print('Resized Dimensions : ',resized.shape)
    
    cv2.imshow("Resized image", resized)
    cv2.imshow("INTER_NEAREST image", resized1)
    cv2.imshow("INTER_LINEAR image", resized2)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# 参考博客：
- https://www.jianshu.com/p/21eff39af5ad
- https://github.com/datawhalechina/team-learning/blob/master/%E8%AE%A1%E7%AE%97%E6%9C%BA%E8%A7%86%E8%A7%89%E5%9F%BA%E7%A1%80%EF%BC%9A%E5%9B%BE%E5%83%8F%E5%A4%84%E7%90%86%EF%BC%88%E4%B8%8A%EF%BC%89/Task01%20%E5%9B%BE%E5%83%8F%E6%8F%92%E5%80%BC%E7%AE%97%E6%B3%95.md#14-%E7%AE%97%E6%B3%95%E7%90%86%E8%AE%BA%E4%BB%8B%E7%BB%8D%E4%B8%8E%E6%8E%A8%E8%8D%90